In [62]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [63]:
import os
import numpy as np
import sys

In [64]:
sys.path.append("/home/lemma/Documents/MAC-POSTS/side_project/network_builder")

In [65]:
from models import *
import MNMAPI
from MNM_mcnb import *

In [66]:
target_time = 3600
early_penalty = 0.7
late_penalty = 10

In [67]:
s = driving_route(target_time, early_penalty, late_penalty, [0,0,0], [1, 2, 4], 1, 3, 120)

In [90]:
single_driving_config = dict()
single_driving_config['mode'] = 'single_drive'
single_driving_config['target_time'] = target_time
single_driving_config['early_penalty'] = early_penalty
single_driving_config['late_penalty'] = late_penalty
single_driving_config['ID_list'] = [0, 0, 0]
single_driving_config['O'] = 0
single_driving_config['D'] = 1
single_driving_config['link_list'] = [link(), link(), link()]
single_driving_config['path_ID'] = 0
single_driving_config['number_people'] = 1
single_driving_config['parking_lot'] = parking_lot(5)
single_driving_config['walking_time'] = 3
p = make_path(single_driving_config)

Creating route for: single_drive


In [93]:
multiple_driving_config = dict()
multiple_driving_config['mode'] = 'multiple_drive'
multiple_driving_config['target_time'] = target_time
multiple_driving_config['early_penalty'] = early_penalty
multiple_driving_config['late_penalty'] = late_penalty
multiple_driving_config['ID_list'] = [0, 0, 0]
multiple_driving_config['O'] = 0
multiple_driving_config['D'] = 1
multiple_driving_config['link_list'] = [link(), link(), link()]
multiple_driving_config['path_ID'] = 0
multiple_driving_config['number_people'] = 2
multiple_driving_config['parking_lot'] = parking_lot(5)
multiple_driving_config['walking_time'] = 3
p = make_path(multiple_driving_config)

Creating route for: multiple_drive


In [70]:
a = MNMAPI.mcdta_api()

## Test NB

In [126]:
data_folder = '../data/input_files_small_multiclass'

In [127]:
nb = MNM_network_builder()

MNM_config
MNM_pathtable


In [128]:
nb.load_from_folder(data_folder)

No demand input
No route choice portition for path table


In [129]:
num_assign_intervals = nb.config.config_dict['DTA']['max_interval']

In [130]:
demand_dict = dict()
demand_dict[1] = dict()
demand_dict[2] = dict()
demand_dict[3] = dict()
demand_dict[4] = dict()
demand_dict[5] = dict()
demand_dict[1][1] = [np.ones(num_assign_intervals) * 10, np.zeros(num_assign_intervals)]
demand_dict[2][1] = [np.zeros(num_assign_intervals), np.ones(num_assign_intervals) * 1]
demand_dict[3][1] = [np.ones(num_assign_intervals) * 10, np.zeros(num_assign_intervals)]
demand_dict[4][1] = [np.ones(num_assign_intervals) * 10, np.zeros(num_assign_intervals)]
demand_dict[5][1] = [np.zeros(num_assign_intervals), np.ones(num_assign_intervals) * 10]
demand_dict[1][2] = [np.ones(num_assign_intervals) * 10, np.zeros(num_assign_intervals)]
demand_dict[1][3] = [np.ones(num_assign_intervals) * 10, np.zeros(num_assign_intervals)]
demand_dict[3][2] = [np.ones(num_assign_intervals) * 10, np.zeros(num_assign_intervals)]
demand_dict[4][3] = [np.ones(num_assign_intervals) * 10, np.zeros(num_assign_intervals)]

In [131]:
choice_dict = dict()
choice_dict = dict()
choice_dict[1] = dict()
choice_dict[2] = dict()
choice_dict[3] = dict()
choice_dict[4] = dict()
choice_dict[5] = dict()
choice_dict[1][1] = 0
choice_dict[2][1] = 1
choice_dict[3][1] = 0
choice_dict[4][1] = 0
choice_dict[5][1] = 1
choice_dict[1][2] = 0
choice_dict[1][3] = 0
choice_dict[3][2] = 0
choice_dict[4][3] = 0

In [132]:
nb.path_table.ID2path

OrderedDict([(0, MNM_path, path ID 0, O node: 101, D node 105),
             (1, MNM_path, path ID 1, O node: 101, D node 105),
             (2, MNM_path, path ID 2, O node: 102, D node 105),
             (3, MNM_path, path ID 3, O node: 103, D node 105),
             (4, MNM_path, path ID 4, O node: 103, D node 105),
             (5, MNM_path, path ID 5, O node: 104, D node 105),
             (6, MNM_path, path ID 6, O node: 104, D node 105),
             (7, MNM_path, path ID 7, O node: 108, D node 105),
             (8, MNM_path, path ID 8, O node: 101, D node 106),
             (9, MNM_path, path ID 9, O node: 101, D node 106),
             (10, MNM_path, path ID 10, O node: 101, D node 107),
             (11, MNM_path, path ID 11, O node: 101, D node 107),
             (12, MNM_path, path ID 12, O node: 103, D node 106),
             (13, MNM_path, path ID 13, O node: 104, D node 107)])

In [133]:
def get_uniform_mcflow(demand_dict, path_dict, ID2path, MNM_od, num_assign_intervals):
    f_car = np.ones((len(ID2path), num_assign_intervals))
    f_truck = np.ones((len(ID2path), num_assign_intervals))
    for O_node in path_dict.keys():
        O = MNM_od.O_dict.inv[O_node]
        for D_node in path_dict[O_node].keys():
            D = MNM_od.D_dict.inv[D_node]
            path_list = path_dict[O_node][D_node].path_list
            num_path = len(path_list)
            for path in path_list:
                idx = ID2path.keys().index(path.path_ID)
                f_car[idx, :] = demand_dict[O][D][0] / np.float(num_path)
                f_truck[idx, :] = demand_dict[O][D][1] / np.float(num_path)
    return f_car.flatten(order = "F"), f_truck.flatten(order = "F")

In [134]:
f_car, f_truck = get_uniform_mcflow(demand_dict, nb.path_table.path_dict, nb.path_table.ID2path, nb.od, num_assign_intervals)

In [135]:
nb.update_demand_path(f_car, f_truck, choice_dict)

In [136]:
nb.dump_to_folder("test")